In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [2]:
import tensorflow as tf
from datasets import load_dataset
from transformers import TFMT5ForConditionalGeneration, MT5Tokenizer, DataCollatorForSeq2Seq
from tensorflow.keras.optimizers import Adam


In [3]:
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small", use_fast=False)
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

NameError: name 'AutoTokenizer' is not defined

In [2]:
from datasets import Dataset, DatasetDict
import pandas as pd
import csv

train_df = pd.read_csv("/content/translated_train.csv", sep=None, engine="python", on_bad_lines='skip')
validation_df = pd.read_csv("/content/translated_test.csv", sep=None, engine="python", on_bad_lines='skip')



# Pandas DataFrame'leri Hugging Face Dataset formatına çevirme
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)

# 📌 Eğitim veri setinin tamamını kullanmak istersen:
# combined_datasets = DatasetDict({
#     "train": train_dataset,
#     "test": test_dataset
# })

# Hugging Face DatasetDict oluşturma
combined_datasets = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset
})
# Mevcut sütun isimlerini kontrol et
print("Validation dataset columns:", combined_datasets["validation"].column_names)

# Sütun ismini değiştirmek için:
combined_datasets["validation"] = combined_datasets["validation"].rename_column("\ufeffarticle", "article")

# Tekrar kontrol et
print("Updated Validation dataset columns:", combined_datasets["validation"].column_names)


# Sonuçları kontrol et
print(combined_datasets)

'''from statistics import mean

# Train datasetindeki article ve highlights sütunlarını al
articles = combined_datasets["train"]["article"]
highlights = combined_datasets["train"]["highlights"]

# Her bir metni tokenize edip uzunluklarını hesapla
article_lengths = [len(tokenizer.tokenize(text)) for text in articles]
highlight_lengths = [len(tokenizer.tokenize(text)) for text in highlights]

# Ortalama uzunluğu hesapla
avg_article_length = round(mean(article_lengths))
avg_highlight_length = round(mean(highlight_lengths))

print(f"📌 Ortalama Article Uzunluğu: {avg_article_length} token")
print(f"📌 Ortalama Highlights Uzunluğu: {avg_highlight_length} token")'''


Validation dataset columns: ['\ufeffarticle', 'highlights']
Updated Validation dataset columns: ['article', 'highlights']
DatasetDict({
    train: Dataset({
        features: ['article', 'highlights'],
        num_rows: 19519
    })
    validation: Dataset({
        features: ['article', 'highlights'],
        num_rows: 1400
    })
})


'from statistics import mean\n\n# Train datasetindeki article ve highlights sütunlarını al\narticles = combined_datasets["train"]["article"]\nhighlights = combined_datasets["train"]["highlights"]\n\n# Her bir metni tokenize edip uzunluklarını hesapla\narticle_lengths = [len(tokenizer.tokenize(text)) for text in articles]\nhighlight_lengths = [len(tokenizer.tokenize(text)) for text in highlights]\n\n# Ortalama uzunluğu hesapla\navg_article_length = round(mean(article_lengths))\navg_highlight_length = round(mean(highlight_lengths))\n\nprint(f"📌 Ortalama Article Uzunluğu: {avg_article_length} token")\nprint(f"📌 Ortalama Highlights Uzunluğu: {avg_highlight_length} token")'

In [1]:
import numpy as np
def tokenize_sample_data(data):
    try:
        # 🔹 Boş verileri kontrol et
        if not data["article"] or not data["highlights"]:
            return None  # Eğer boşsa, bu satırı atla

        # 🔹 Article (Girdi) için tokenizasyon
        input_feature = tokenizer(
            data["article"],
            truncation=True,
            max_length=1024,  # 1150 yerine 1024'e düşürüyoruz (daha güvenli)
            padding="max_length"
        )

        # 🔹 Highlights (Çıktı) için tokenizasyon - MT5 için özel olarak
        with tokenizer.as_target_tokenizer():
            label = tokenizer(
                data["highlights"],
                truncation=True,
                max_length=128,
                padding="max_length"
            )

        # 🔹 Token listesi boş mu kontrol et
        if len(input_feature["input_ids"]) == 0 or len(label["input_ids"]) == 0:
            print(f"⚠️ Boş token listesi bulunan veri: {data}")
            return None  # Hatalı satırları atla

        # 🔹 Token ID'leri sınırını kontrol et (MT5 için vocab size kontrolü)
        if max(input_feature["input_ids"]) >= tokenizer.vocab_size or max(label["input_ids"]) >= tokenizer.vocab_size:
            print(f"⚠️ Token ID sınır dışı: Girdi max={max(input_feature['input_ids'])}, Çıktı max={max(label['input_ids'])}, Vocab size={tokenizer.vocab_size}")
            return None  # Sınır dışı token ID'leri içeren verileri atla

        # 🔹 Padding token'ları -100 ile değiştir (loss hesaplanmaması için)
        labels = np.array(label["input_ids"])
        # MT5 için pad token genellikle 0'dır, kontrol ederek değiştirin
        pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0
        labels[labels == pad_token_id] = -100

        return {
            "input_ids": input_feature["input_ids"],
            "attention_mask": input_feature["attention_mask"],
            "labels": labels.tolist(),  # numpy array'i listeye çevir
        }
    except Exception as e:
        print(f"⚠️ Hata veren veri: {data}\nHata: {str(e)}")
        return None  # Hata varsa satırı atla


# 🔹 Veriyi tokenize et, hatalı satırları çıkar
tokenized_ds = combined_datasets.map(
    tokenize_sample_data,
    batched=False,  # 🔥 Tek tek satır işleyerek hata yakalayalım
    remove_columns=combined_datasets["train"].column_names  # Orijinal sütunları kaldır
)


NameError: name 'combined_datasets' is not defined

In [9]:
example_text = "Ne desem bilemedim insanlar çok garip. Jackson bir numaralı ajanımız"
tokenized_example = tokenizer(example_text)

print(tokenized_example["input_ids"])
print(f"Max token ID: {max(tokenized_example['input_ids'])}, Tokenizer vocab size: {tokenizer.vocab_size}")


[1648, 498, 470, 3871, 221201, 29425, 259, 3803, 121135, 325, 260, 32614, 698, 79878, 2555, 44311, 40885, 1]
Max token ID: 221201, Tokenizer vocab size: 250100


In [10]:
# 🔹 Boş veya hatalı satırları bul
def check_validation_set(dataset):
    for i, data in enumerate(dataset):
        try:
            if not data["article"] or not data["highlights"]:
                print(f"⚠️ Boş satır bulundu (Index: {i}): {data}")

            # 🔹 Tokenizer ile kontrol et
            tokenized = tokenizer(data["article"], truncation=True, max_length=1150)
            if len(tokenized["input_ids"]) == 0:
                print(f"⚠️ Boş token listesi! (Index: {i}): {data['article']}")
        except Exception as e:
            print(f"⚠️ Hata veren veri (Index {i}): {data}\nHata: {str(e)}")

# Validation seti kontrol et
check_validation_set(combined_datasets["validation"])


In [11]:
def clean_validation_set(dataset):
    cleaned_data = []
    for data in dataset:
        try:
            if not data["article"] or not data["highlights"]:
                continue  # Boş satırları atla

            # Tokenizasyonu dene
            tokenized = tokenizer(data["article"], truncation=True, max_length=1150)
            if len(tokenized["input_ids"]) == 0:
                continue  # Tokenizasyon başarısızsa atla

            cleaned_data.append(data)
        except Exception:
            continue  # Hata varsa satırı atla

    return Dataset.from_list(cleaned_data)

# Validation dataset'i temizle
combined_datasets["validation"] = clean_validation_set(combined_datasets["validation"])
print(f"✅ Temizlenmiş validation seti boyutu: {len(combined_datasets['validation'])}")


✅ Temizlenmiş validation seti boyutu: 1400


In [52]:
'''

from transformers import MT5Tokenizer, MT5ForConditionalGeneration

# 1. Tokenizer ve modelin yüklenmesi
model_name = "google/mt5-small"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

# 2. Özetlenmek istenen Türkçe metin
turkish_text = """
İklim değişikliği, küresel sıcaklıkların artışıyla birlikte ortaya çıkan çevresel, sosyal ve ekonomik sorunlara neden olmaktadır.
Özellikle kuraklık, sel ve orman yangınları gibi doğal afetlerin sıklığı artarken, tarımsal üretimde de ciddi düşüşler yaşanmaktadır.
Bu durum, gıda güvenliğini tehdit etmekte ve toplumları olumsuz yönde etkilemektedir.
"""

# 3. Özetleme için giriş formatı: 'summarize:' ön ekini ekliyoruz
input_text = f"summarize: {turkish_text}"

# 4. Tokenizer ile metni encode etme
input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

# 5. Modeli kullanarak özetleme yapma
summary_ids = model.generate(
    input_ids,
    max_length=150,
    min_length=30,
    length_penalty=2.0,
    num_beams=4,
    early_stopping=True
)

# 6. Özetlenen metni decode etme
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 7. Sonucu yazdırma
print("Özetlenen Metin:")
print(summary)
'''

'\n\nfrom transformers import MT5Tokenizer, MT5ForConditionalGeneration\n\n# 1. Tokenizer ve modelin yüklenmesi\nmodel_name = "google/mt5-small"\ntokenizer = MT5Tokenizer.from_pretrained(model_name)\nmodel = MT5ForConditionalGeneration.from_pretrained(model_name)\n\n# 2. Özetlenmek istenen Türkçe metin\nturkish_text = """\nİklim değişikliği, küresel sıcaklıkların artışıyla birlikte ortaya çıkan çevresel, sosyal ve ekonomik sorunlara neden olmaktadır.\nÖzellikle kuraklık, sel ve orman yangınları gibi doğal afetlerin sıklığı artarken, tarımsal üretimde de ciddi düşüşler yaşanmaktadır.\nBu durum, gıda güvenliğini tehdit etmekte ve toplumları olumsuz yönde etkilemektedir.\n"""\n\n# 3. Özetleme için giriş formatı: \'summarize:\' ön ekini ekliyoruz\ninput_text = f"summarize: {turkish_text}"\n\n# 4. Tokenizer ile metni encode etme\ninput_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)\n\n# 5. Modeli kullanarak özetleme yapma\nsummary_ids = model.genera

In [13]:
'''import torch
from transformers import AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modeli yükle (önceden tanımladığın tokenizer'ı kullan)
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small").to(device)
'''

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [14]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    padding="longest",  # "max_length" yerine "longest" kullanın
    return_tensors="pt",
)

In [15]:
!pip install evaluate
!pip install rouge_score

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=fb7991385c694bc9572497993bc175939a5a9ce413e91ed8853adf6439b008ee
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [16]:
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer

rouge_metric = evaluate.load("rouge")

# define function for custom tokenization
def tokenize_sentence(arg):
    return arg  # Bu fonksiyon kaldırılabilir veya değiştirilmeden kullanılabilir

# define function to get ROUGE scores with custom tokenization
def metrics_func(eval_arg):
  preds, labels = eval_arg
  # Replace -100
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  # Convert id tokens to text
  text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  # Insert a line break (\n) in each sentence for ROUGE scoring
  # (Note : Please change this code, when you perform on other languages except for Japanese)
  text_preds = [(p if p.endswith((".", "!", "?")) else p + ".") for p in text_preds]
  text_labels = [(l if l.endswith((".", "!", "?")) else l + ".") for l in text_labels]

  # Türkçe cümle tokenizasyonu
  sent_tokenizer_tr = RegexpTokenizer(r'[^.!?]*[.!?]')
  text_preds = ["\n".join(np.char.strip(sent_tokenizer_tr.tokenize(p))) for p in text_preds]
  text_labels = ["\n".join(np.char.strip(sent_tokenizer_tr.tokenize(l))) for l in text_labels]

  # compute ROUGE score with custom tokenization
  return rouge_metric.compute(
    predictions=text_preds,
    references=text_labels,
    tokenizer=tokenize_sentence
  )

Text 0: Memphis, Cuma günü Oklahoma City karşısında aldığı...
Tokens 0: [259, 127630, 261, 7373, 262, 259, 29889, 259, 48311, 2740, 11993, 18712, 440, 11693, 966, 89041, 277, 1185, 4822, 107739, 15913, 44016, 259, 103184, 276, 189744, 99520, 273, 277, 3841, 259, 266, 39846, 17010, 739, 96001, 720, 529, 489, 158281, 36400, 107859, 8121, 90861, 277, 2432, 259, 85945, 2186, 259, 63027, 4300, 60697, 277, 273, 172818, 1176, 260, 489, 158281, 36400, 261, 259, 103184, 276, 189744, 99520, 273, 277, 3841, 49330, 2222, 1099, 10946, 62435, 12018, 31079, 529, 1001, 259, 195164, 269, 156890, 259, 1907, 90056, 126584, 8534, 468, 11454, 3143, 18966, 6765, 277, 348, 18629, 14987, 108206, 260, 1716, 13883, 261, 259, 70444, 277, 2186, 36591, 11351, 259, 135005, 17010, 492, 75593, 356, 79878, 739, 4919, 259, 48431, 7757, 78159, 261, 259, 105339, 277, 407, 11110, 5422, 38892, 14514, 96362, 278, 13904, 494, 259, 48431, 5372, 81904, 261, 42601, 11993, 18712, 440, 11693, 742, 31001, 4822, 107739, 15913, 4822

In [19]:
print(validation_df.isnull().sum())  # Boş hücreleri say
print(validation_df.head())          # İlk satırları göster




﻿article      0
highlights    0
dtype: int64
                                            ﻿article  \
0  Memphis, Cuma günü Oklahoma City karşısında al...   
1  Genellikle evde filtre kahve demlerken kahve t...   
2  Ronald Reagan'ı vuran adamı tedavi eden bir ps...   
3  Durham'dan 23 yaşındaki Lix Bussey, erkek arka...   
4  Araştırmacılar, beynin 'nötr' veya sıkıcı olay...   

                                          highlights  
0  Memphis Grizzles'ın savunması Oklahoma City ka...  
1  Tüy bırakmayan ve yırtılmaya karşı dayanıklı f...  
2  Dr. Giorgi-Guarnieri, Hinckley'nin akıl hastan...  
3  Durham'dan 23 yaşındaki Alix Bussey Meksika ta...  
4  New York Üniversitesi psikologları beynin anıl...  


In [20]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="mt5-summarize-tr",
    log_level="error",
    num_train_epochs=10,  # 15 yerine 10 (overfitting riskini azaltmak için)
    learning_rate=1e-6,  # 3e-5 fazla olabilir, 2e-5 daha stabil
    lr_scheduler_type="linear",
    warmup_steps=800,  # 1000 yerine 800 (hızlı stabilizasyon için)
    optim="adafactor",
    weight_decay=0.01,
    per_device_train_batch_size=8,  # Batch size'ı küçültün
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=16
    evaluation_strategy="steps",
    eval_steps=200,  # 250 yerine 200 (erken durdurma için daha sık değerlendirme)
    predict_with_generate=True,
    generation_max_length=256,
    save_steps=1000,  # 2000 yerine 1000 (daha sık model kaydı)
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_total_limit=3,  # En iyi 3 modeli sakla, disk kullanımını optimize et
    report_to="none",  # WandB veya TensorBoard istemiyorsan "none"
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
for i, seq in enumerate(tokenized_ds["validation"]["labels"]):
    for token_id in seq:
        if token_id >= tokenizer.vocab_size or token_id < 0:
            print(f"Hatalı Token ID Bulundu! Index: {i}, Token ID: {token_id}")


In [22]:
print(f"Tokenizer: {tokenizer.name_or_path}")
print(f"Model: {model.config._name_or_path}")


Tokenizer: google/mt5-small
Model: google/mt5-small


In [23]:
from transformers import Seq2SeqTrainer, EarlyStoppingCallback

# EarlyStoppingCallback ile erken durdurma tanımlıyoruz
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=5,  # Art arda kaç kez validation loss'un düşmemesine izin verilecek
    early_stopping_threshold=0.0  # Minimum değişiklik miktarı (opsiyonel)
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=metrics_func,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"],
    callbacks=[early_stopping]  # Erken durdurma callback'i buraya ekleniyor
)

trainer.train()


Step,Training Loss,Validation Loss


IndexError: piece id is out of range.

In [ ]:
# Eğitilen modeli kaydet
output_dir = "./trained_model"  # Kaydedilecek dizin
trainer.save_model(output_dir)  # Modeli ve tokenizer'ı kaydeder

# Tokenizer'ı ayrıca kaydetmek isterseniz
tokenizer.save_pretrained(output_dir)

print(f"Model ve tokenizer {output_dir} dizinine kaydedildi.")

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Eğitilen modeli ve tokenizer'ı yükle
model = AutoModelForSeq2SeqLM.from_pretrained("./trained_model")
tokenizer = AutoTokenizer.from_pretrained("./trained_model")


In [ ]:
test_text = "Dünyamızda 1900’lü yıllardan günümüze kadar dil öğretiminde çeşitli metinler kullanılmıştır. Bunlar “ edebi metinler, üretilmiş metinler, özgün ve özel metinler ” başlıkları altında toplanmıştır. Metinlerin seçimi dil öğretim yaklaşım ve yöntemlerine göre değişmektedir. Her yaklaşım kendine özgü metin kullanmıştır. Geleneksel yaklaşımda dil bilgisi kuralları, atasözleri, edebiyat, genel kültür gibi konuların öğretimine ağırlık verildiğinden edebi metinler kullanılmıştır. Davranışçı yaklaşımda dil davranış olarak ele alınmış, tekrar, taklit ve ezberleme yoluyla öğretilmiştir. Bu yaklaşımda edebi metinler yerine üretilmiş metinler kullanılmıştır. Bilişsel yaklaşımda “dil iletişim aracıdır” görüşü yayılmış ve özgün metinler kullanılmaya başlanmıştır. "


In [ ]:
'''inputs = tokenizer(
    test_text,
    return_tensors="pt",  # PyTorch tensörleri olarak döner
    max_length=1024,      # Modelin giriş sınırı
    truncation=True       # Çok uzun metinleri keser
)
'''

In [ ]:
# Özetleme
output_ids = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_length=128,  # Özet uzunluğu sınırı
    num_beams=4,     # Beam search kullanımı
    no_repeat_ngram_size=2,  # Tekrar eden n-gramları engeller
    length_penalty=0.6       # Kısa özetlere öncelik
)

# Özet çözümleme
summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Özet:", summary)


In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''